In [1]:
import pandas as pd
import os
import pickle
import fasttext
import re

In [ ]:
# Load data
DATA_PATH = './data/'
pickle_file = os.path.join(DATA_PATH, 'first_data_batch.pkl')
df = pd.read_pickle(pickle_file)

In [3]:
def preprocess_hebrew(text):
    # Remove backslash-prefixed tags (e.g., \n, \t)
    text = re.sub(r"\\[a-zA-Z]+", "", text)

    # Remove unwanted control characters (whitespace + control characters + BOM)
    text = re.sub(r'[\x00-\x1F\x7F-\x9F\u00A0]+', '', text)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    return text

In [4]:
# Apply preprocessing to Hebrew text
df['headline'] = df['headline'].apply(preprocess_hebrew)
df['combined_info'] = df['combined_info'].apply(preprocess_hebrew)

In [6]:
speeches = df.loc[(df["combined_info"] != "")][['book_id', 'type', 'unit', 'headline', 'combined_info']]
speeches[:5]

,book_id,type,unit,headline,combined_info
6,241112,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"חזון עתיד מתוקן, חגי ישראל, חזון הנביאים, תנופ...","ד. בן-גוריון י""ח אייר תש""ט 17.5.49אין עמים ..."
9,241170,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"israel and the middle east, חברות ישראל באו""ם,...",For Israel and Middle EastBy David Ben-Gurion ...
16,241180,ארכיון בן גוריון - מסמך,נאומים ומאמרים,חוק שירות הביטחון,"ישיבה ע' ו, 5.9.1949 חוק שירות הבטחון תש""ט-..."
23,241225,ארכיון בן גוריון - מסמך,נאומים ומאמרים,how ambassador Morgenthau saved Palestine jewr...,HOW AMBASSADOR MORGENTHAU SAVED PALESTINE JEWR...
68,241459,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"[ח""א?] במטה המורחב, צה""ל, מערכת סיני, מלחמת סי...","12.12.57 ח""א במטה המורחבאני מודה, שהדברים של מ..."


In [ ]:
# Load the pretrained language detection model
model_path = os.path.join(DATA_PATH, 'lid.176.bin')
model = fasttext.load_model(model_path)

# Define a function to detect the language of a given text
def detect_lang(text):
    if not isinstance(text, str) or text.strip() == "":
        return "unknown"  
   
    label = model.predict(text)[0][0]  
    return label.replace("__label__", "")

# Apply language detection
speeches['lang'] = speeches['combined_info'].apply(lambda x: detect_lang(x) if isinstance(x, str) else 'unknown')

# Filter out non-Hebrew and non-English speeches
speeches = speeches[speeches['lang'].isin(['he', 'en'])]

In [8]:
speeches = speeches.reset_index(drop=True)

In [9]:
new_data_path = os.path.join(DATA_PATH, "prepd_data.pkl")
with open(new_data_path, "wb") as file: 
    pickle.dump(speeches, file)